In [1]:
!pip install bing-image-downloader

In [2]:
from bing_image_downloader import downloader
downloader.download("sunflower", limit=100, output_dir='train_images', adult_filter_off=True)
downloader.download("sunflower", limit=20, output_dir='test_images', adult_filter_off=True)
downloader.download("watch", limit=100, output_dir='train_images', adult_filter_off=True)
downloader.download("watch", limit=20, output_dir='test_images', adult_filter_off=True)
downloader.download("deer", limit=100, output_dir='train_images', adult_filter_off=True)
downloader.download("deer", limit=20, output_dir='test_images', adult_filter_off=True)


[%] Downloading Images to /content/train_images/sunflower


[!!]Indexing page: 1

[%] Indexed 100 Images on Page 1.


[%] Downloading Image #1 from https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1200px-Sunflower_sky_backdrop.jpg
[Error]Invalid image, not saving https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1200px-Sunflower_sky_backdrop.jpg

[!] Issue getting: https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1200px-Sunflower_sky_backdrop.jpg
[!] Error:: Invalid image, not saving https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Sunflower_sky_backdrop.jpg/1200px-Sunflower_sky_backdrop.jpg

[%] Downloading Image #1 from https://www.croatia-times.com/wp-content/uploads/2017/07/Sunflowers-of-Slavonia-Emica-Elvedji-HANZA-MEDIA-10.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://content.thriveglobal.in/wp-content/uploads/sites/3/2019/12/sunflower-field-36209

In [3]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np

In [4]:
train_dataset_path = '/content/train_images'
test_dataset_path = '/content/test_images'

In [5]:
mean = [0.4363, 0.4328, 0.3291]
std = [0.2129, 0.2075, 0.2038]

train_transforms = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))                              
])

In [6]:
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_path, transform=train_transforms)
test_dataset = torchvision.datasets.ImageFolder(root=test_dataset_path, transform=test_transforms)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  return torch.device(dev)

In [43]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
  device = set_device()

  for epoch in range(n_epochs):
    print("Epoch number %d " % (epoch + 1))
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    total = 0

    for data in train_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)

      optimizer.zero_grad()

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      running_correct += (labels==predicted).sum().item()

    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100.00 * running_correct / total

    print("  -Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f" % (running_correct, total, epoch_acc, epoch_loss))

    evaluate_model(model, test_loader)

  print("Finished")
  return model

In [44]:
def evaluate_model(model, test_loader):
  model.eval()
  predicted_correctly_on_epoch = 0
  total = 0
  device = set_device()

  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      predicted_correctly_on_epoch += (predicted == labels).sum().item()

    epoch_acc = 100.0 * predicted_correctly_on_epoch / total

    print("  -Testing dataset. Got %d out of %d images correctly (%.3f%%)" % (predicted_correctly_on_epoch, total, epoch_acc))

    

In [45]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


resnet18_model = models.resnet18(pretrained=False)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 3
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
device = set_device()
resnet_18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [47]:
train_nn(resnet18_model, train_loader, test_loader, loss_fn, optimizer, 20)

Epoch number 1 


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


  -Training dataset. Got 255 out of 299 images correctly (85.284%). Epoch loss: 0.346
  -Testing dataset. Got 57 out of 60 images correctly (95.000%)
Epoch number 2 
  -Training dataset. Got 261 out of 299 images correctly (87.291%). Epoch loss: 0.283
  -Testing dataset. Got 55 out of 60 images correctly (91.667%)
Epoch number 3 
  -Training dataset. Got 269 out of 299 images correctly (89.967%). Epoch loss: 0.270
  -Testing dataset. Got 58 out of 60 images correctly (96.667%)
Epoch number 4 
  -Training dataset. Got 273 out of 299 images correctly (91.304%). Epoch loss: 0.218
  -Testing dataset. Got 58 out of 60 images correctly (96.667%)
Epoch number 5 
  -Training dataset. Got 269 out of 299 images correctly (89.967%). Epoch loss: 0.250
  -Testing dataset. Got 59 out of 60 images correctly (98.333%)
Epoch number 6 
  -Training dataset. Got 276 out of 299 images correctly (92.308%). Epoch loss: 0.339
  -Testing dataset. Got 45 out of 60 images correctly (75.000%)
Epoch number 7 
  -T

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  